In [117]:
import pandas as pd
import numpy as np
import re

In [118]:
data = pd.read_csv('thanksgiving.csv', encoding="Latin-1")

In [119]:
data.head(3)

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic
1,4337951949,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Other (please specify),Homemade cranberry gelatin ring,...,No,No,Yes,No,NaN,Rural,18 - 29,Female,"$50,000 to $74,999",East South Central
2,4337935621,Yes,Turkey,NaN,Roasted,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,No,NaN,Suburban,18 - 29,Male,"$0 to $9,999",Mountain


In [120]:
data.columns

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)',
       'Do you typically have gravy?',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       'Which of these side dishes aretypically served

###### Separating the people that celebrate thanksgiving for further analysis

In [121]:
count_celebrate = data['Do you celebrate Thanksgiving?'].value_counts()

In [122]:
count_celebrate

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [123]:
data_celebrate = data[data['Do you celebrate Thanksgiving?'] == 'Yes']

###### counting the number of the unique main dishes people eat in tnxgiving dinner

In [124]:
data_celebrate['What is typically the main dish at your Thanksgiving dinner?'].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

###### Checking if the people who eat Tofurkey, have gravy as well or not?

In [125]:
#data_celebrate[['What is typically the main dish at your Thanksgiving dinner?','Do you typically have gravy?']][data_celebrate['What is typically the main dish at your Thanksgiving dinner?']=='Tofurkey']

###### checking if a person ate one of  the  Apple, Pumpkin, Pecan for desert 

In [126]:
ate_pies = data_celebrate[pd.isnull(data_celebrate['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple']) &\
                         pd.isnull(data_celebrate['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin']) &\
                                        pd.isnull(data_celebrate['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan'])]

In [127]:
#ate_pies

##### analyze the age column
###### replace the average age of each category instead of the range

In [128]:
age_categs = data_celebrate['Age'].value_counts()
age_categs

45 - 59    269
60+        258
30 - 44    235
18 - 29    185
Name: Age, dtype: int64

In [129]:
def transform_age(age):
    if pd.isnull(age):
        return None
    elif age == '60+':
        age_parsed = age[0:2]
        return int(age_parsed)
    else:
        age_parsed = age.split(' ')
        #print((int(age_parsed[0])+int(age_parsed[2]))/2)
        mean_age = (int(age_parsed[0])+int(age_parsed[2]))/2
        return mean_age

In [130]:
data_age_tranformed = data_celebrate['Age'].apply(transform_age)

In [131]:
data_age_tranformed.describe()

count    947.000000
mean      44.889652
std       13.475203
min       23.500000
25%       37.000000
50%       52.000000
75%       60.000000
max       60.000000
Name: Age, dtype: float64

### conclusion: this cannot be a good age analysis, since only the age range was provided

###### convert the houshold earnings to numerical values and get the statistical data

In [132]:
data_celebrate['How much total combined money did all members of your HOUSEHOLD earn last year?'].value_counts()
data_celebrate['How much total combined money did all members of your HOUSEHOLD earn last year?'].unique()

array(['$75,000 to $99,999', '$50,000 to $74,999', '$0 to $9,999',
       '$200,000 and up', '$100,000 to $124,999', '$25,000 to $49,999',
       'Prefer not to answer', '$10,000 to $24,999',
       '$175,000 to $199,999', '$150,000 to $174,999',
       '$125,000 to $149,999', nan], dtype=object)

In [133]:
def transform_earnings(earn):
    if pd.isnull(earn):
        return None
    elif earn =='Prefer not to answer':
        return None
    elif earn == '$200,000 and up':
        return 200000
    else:
        earn_split = earn.split(' ')
        earning_list = []
        if earn_split[0] != '$0':
            for item in [earn_split[0], earn_split[2]]:
                earning = item[1:]
                m = re.search(',',earning)
                earning = earning[:m.start()] + earning[m.end():]
                earning_list.append(int(earning))
        else:
            earning_list.append(0)
            earning = earn_split[2][1:]
            m = re.search(',',earning)
            earning = earning[:m.start()] + earning[m.end():]
            earning_list.append(int(earning))
        earning_mean = sum(earning_list)/2
        return earning_mean 

In [134]:
data_earnings = data_celebrate['How much total combined money did all members of your HOUSEHOLD earn last year?'].apply(transform_earnings)

In [135]:
data_earnings.describe()

count       829.000000
mean      86486.276840
std       57789.467567
min        4999.500000
25%       37499.500000
50%       87499.500000
75%      112499.500000
max      200000.000000
Name: How much total combined money did all members of your HOUSEHOLD earn last year?, dtype: float64